In [1]:
import pandas as pd

In [2]:
data_path = './corona_scotland.xlsx'

# Cases and Death Data

In [3]:
df = pd.read_excel(data_path, 
                   dtype={'Total Tests': 'Int32',
                          'Total Positive Cases': 'Int32',
                          'Total Deaths': 'Int32'}
                   ).ffill()
df = df[df['Health Board'] != 'Golden Jubilee National Hospital']

In [4]:
df

,Date,Health Board,Number of Positive cases,Total Tests,Total Positive Cases,Total Deaths
0,2020-03-06,Tayside,1,1514,11,0
1,2020-03-06,Ayrshire & Arran,1,1514,11,0
2,2020-03-06,Forth Valley,2,1514,11,0
3,2020-03-06,Greater Glasgow and Clyde,1,1514,11,0
4,2020-03-06,Grampian,3,1514,11,0
...,...,...,...,...,...,...
856,2020-05-09,Lothian,2324,71092,13305,1847
857,2020-05-09,Orkney,7,71092,13305,1847
858,2020-05-09,Shetland,54,71092,13305,1847
859,2020-05-09,Tayside,1486,71092,13305,1847


## clean data

In [5]:
df['Health Board'].value_counts()

Greater Glasgow and Clyde        65
Lothian                          65
Fife                             65
Tayside                          65
Forth Valley                     65
Grampian                         65
Lanarkshire                      64
Shetland                         62
Borders                          60
Highland                         57
Dumfries and Galloway            55
Ayrshire and Arran               42
Eileanan Siar (Western Isles)    39
Orkney                           39
Ayrshire and Arran               20
Ayrshire & Arran                  3
Name: Health Board, dtype: int64

In [6]:
# clean up some names
df.loc[df['Health Board'] == 'Ayrshire\xa0and Arran', 'Health Board'] = 'Ayrshire and Arran' 
df.loc[df['Health Board'] == 'Ayrshire & Arran', 'Health Board'] = 'Ayrshire and Arran' 

In [7]:
df['Health Board'].value_counts()

Greater Glasgow and Clyde        65
Lothian                          65
Fife                             65
Tayside                          65
Forth Valley                     65
Ayrshire and Arran               65
Grampian                         65
Lanarkshire                      64
Shetland                         62
Borders                          60
Highland                         57
Dumfries and Galloway            55
Eileanan Siar (Western Isles)    39
Orkney                           39
Name: Health Board, dtype: int64

## table of number cases per region over time

In [8]:
df_cases = df[['Date', 'Health Board', 'Number of Positive cases']]

In [9]:
df_table = pd.pivot_table(df_cases, 
                          columns=['Health Board'], 
                          values=['Number of Positive cases'], 
                          index=['Date'],
                          fill_value=0,
                         ).astype('Int32')

df_table.columns = df_table.columns.droplevel(0)

In [10]:
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [11]:
df_table.to_pickle('corona_scotland_regions_ts.pk')

## table of national cases, tests, and deaths over time

In [12]:
df_national = (df.drop(columns=['Health Board', 'Number of Positive cases'])
                 .groupby('Date').first()
              )

In [13]:
df_national.index

DatetimeIndex(['2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
               '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
               '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
               '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
               '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',
               '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
               '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
               '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06',
               '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10',
               '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14',
               '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18',
               '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22',
               '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26',
               '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30',
      

In [14]:
df_national.to_pickle('corona_scotland_total_stats.pk')

## Scotland Population Data

In [15]:
region_pops = {
    'Ayrshire and Arran': 369670,
    'Borders': 115270, 
    'Dumfries and Galloway': 148790,
    'Fife': 371910,
    'Forth Valley': 306070,
    'Grampian': 584550,
    'Greater Glasgow and Clyde': 1196335,
    'Highland': 321800,
    'Lanarkshire': 563185,
    'Lothian': 897770,
    'Orkney': 22190,
    'Shetland': 22990,
    'Tayside': 416080,
    'Eileanan Siar (Western Isles)': 26500,
    #'Scotland (Total)': 5438000,
}

In [16]:
df_pop = pd.DataFrame(region_pops.items(), columns=['Region', 'Pop']).set_index('Region').sort_index()

In [17]:
df_pop.loc['Scotland (Total)'] = 5438000

In [18]:
df_pop

,Pop
Region,
Ayrshire and Arran,369670
Borders,115270
Dumfries and Galloway,148790
Eileanan Siar (Western Isles),26500
Fife,371910
Forth Valley,306070
Grampian,584550
Greater Glasgow and Clyde,1196335
Highland,321800


In [19]:
df_pop[:14].sum()
## hmm missing few thousands??

Pop    5363110
dtype: int64

In [20]:
df_pop.to_pickle('scotland_population.pk')

# Hospital Data

In [21]:
from datetime import datetime

In [22]:
df = pd.read_excel(data_path, 
                   usecols=['date', 'health_board', 
                            'num_hospital', 'num_icu'],
                   dtype={
                       'date': datetime,
                       'health_board': 'str',
                       'num_hospital': 'Int32',
                       'num_icu': 'Int32'},
                   na_values={'num_hospital': '*',
                              'num_icu': '*'},
                   sheet_name=1,
                   )
df['date'].ffill(inplace=True)
df.fillna(0, inplace=True)

In [23]:
df

,date,health_board,num_hospital,num_icu
0,2020-04-08,Ayrshire and Arran,91,16
1,2020-04-08,Borders,56,9
2,2020-04-08,Dumfries and Galloway,44,6
3,2020-04-08,Fife,127,13
4,2020-04-08,Forth Valley,107,9
...,...,...,...,...
474,2020-05-09,Orkney,0,0
475,2020-05-09,Shetland,0,0
476,2020-05-09,Tayside,63,0
477,2020-05-09,Eileanan Siar (Western Isles),0,0


In [24]:
df_hos = df[['date', 'health_board', 'num_hospital']]
df_icu = df[['date', 'health_board', 'num_icu']]

In [25]:
df_table = pd.pivot_table(df_hos, 
                          columns=['health_board'], 
                          values=['num_hospital'], 
                          index=['date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [26]:
df_table.tail()

health_board,Ayrshire and Arran,Borders,Dumfries and Galloway,Eileanan Siar (Western Isles),Fife,Forth Valley,Golden Jubilee National Hospital,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Scotland (Total)
date,,,,,,,,,,,,,,,,
2020-05-05,93,32,21,0,118,81,0,165,523,52,223,274,0,0,64,1646
2020-05-06,108,37,17,0,109,82,0,151,509,37,229,274,0,0,64,1617
2020-05-07,97,38,0,0,100,79,0,135,506,27,212,302,0,0,65,1561
2020-05-08,91,43,22,0,110,75,5,138,502,33,206,287,0,0,63,1575
2020-05-09,91,35,0,0,120,79,0,118,496,35,200,323,0,0,63,1560


In [27]:
df_table.to_pickle('corona_scotland_regions_hospital_ts.pk')

In [28]:
df_table = pd.pivot_table(df_icu, 
                          columns=['health_board'], 
                          values=['num_icu'], 
                          index=['date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [29]:
df_table.tail()

health_board,Ayrshire and Arran,Borders,Dumfries and Galloway,Eileanan Siar (Western Isles),Fife,Forth Valley,Golden Jubilee National Hospital,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Scotland (Total)
date,,,,,,,,,,,,,,,,
2020-05-05,8,0,0,0,5,0,0,13,32,0,10,18,0,0,0,86
2020-05-06,7,0,0,0,0,0,0,11,30,0,5,18,0,0,0,71
2020-05-07,7,0,0,0,0,0,0,11,32,0,7,12,0,0,0,69
2020-05-08,6,0,0,0,0,0,0,11,31,0,6,13,0,0,0,67
2020-05-09,9,0,0,0,0,0,0,10,30,0,8,20,0,0,0,77


In [30]:
df_table.to_pickle('corona_scotland_regions_icu_ts.pk')